In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [ ]:
!pip install huggingface_hub ipywidgets 
! pip install -U "huggingface_hub[cli]"


In [ ]:
from huggingface_hub import notebook_login

notebook_login()    

In [2]:
record = []
x = []

In [ ]:
!pip install rouge_score

In [ ]:
!pip install indic-nlp-library

In [ ]:
!pip install datasets transformers accelerate evaluate sentencepiece

In [ ]:
!pip install sentencepiece

In [3]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs
    num_gpus = torch.cuda.device_count()
    print("Number of available GPUs:", num_gpus)

    # Get the name of each GPU
    for gpu_id in range(num_gpus):
        gpu_name = torch.cuda.get_device_name(gpu_id)
        print("GPU", gpu_id, ":", gpu_name)
else:
    print("CUDA is not available. Make sure you have a GPU and PyTorch with CUDA support installed.")


Number of available GPUs: 1
GPU 0 : Quadro GP100


In [4]:
!export CUDA_VISIBLE_DEVICES=0

In [5]:
from torch import cuda
import torch
device = 'cuda' if cuda.is_available() else 'cpu'
torch.cuda.set_device(0)

In [6]:
import torch

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the index of the currently selected device
    gpu_number = torch.cuda.current_device()
    print("GPU number:", gpu_number)
else:
    print("CUDA is not available. Make sure you have a GPU and PyTorch with CUDA support installed.")


GPU number: 0


## Getting the model and datasets

In [75]:
from datasets import load_dataset

dataset = load_dataset("ILSUM/ILSUM-1.0", 'Hindi')

In [76]:
from transformers import MBartForConditionalGeneration, AutoModelForSeq2SeqLM
from transformers import AlbertTokenizer, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)

# Or use tokenizer = AlbertTokenizer.from_pretrained("ai4bharat/IndicBART", do_lower_case=False, use_fast=False, keep_accents=True)

model = AutoModelForSeq2SeqLM.from_pretrained("ai4bharat/IndicBART")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [77]:

# Or use model = MBartForConditionalGeneration.from_pretrained("ai4bharat/IndicBART")

# Some initial mapping
bos_id = tokenizer._convert_token_to_id_with_added_voc("<s>")
eos_id = tokenizer._convert_token_to_id_with_added_voc("</s>")
pad_id = tokenizer._convert_token_to_id_with_added_voc("<pad>")
# To get lang_id use any of ['<2as>', '<2bn>', '<2en>', '<2gu>', '<2hi>', '<2kn>', '<2ml>', '<2mr>', '<2or>', '<2pa>', '<2ta>', '<2te>']

# First tokenize the input and outputs. The format below is how IndicBART was trained so the input should be "Sentence </s> <2xx>" where xx is the language code. Similarly, the output should be "<2yy> Sentence </s>".
inp = tokenizer("I am a boy </s> <2en>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids # tensor([[  466,  1981,    80, 25573, 64001, 64004]])

out = tokenizer("<2hi> मैं  एक लड़का हूँ </s>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids # tensor([[64006,   942,    43, 32720,  8384, 64001]])
# Note that if you use any language other than Hindi or Marathi, you should convert its script to Devanagari using the Indic NLP Library.
model_outputs=model(input_ids=inp, decoder_input_ids=out[:,0:-1], labels=out[:,1:])

# For loss
print(out)
print(out[:,1:])
out[:,0:-1] ## This is not label smoothed.

# For logits
#model_outputs.logits


tensor([[64006,   942,    43, 32720,  8384, 64001]])
tensor([[  942,    43, 32720,  8384, 64001]])


tensor([[64006,   942,    43, 32720,  8384]])

In [78]:
print(pad_id)

0


In [79]:

# For generation. Pardon the messiness. Note the decoder_start_token_id.

model.eval() # Set dropouts to zero

model_output=model.generate(inp, use_cache=True, num_beams=4, max_length=20, min_length=1, early_stopping=True, pad_token_id=pad_id, bos_token_id=bos_id, eos_token_id=eos_id, decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2en>"))

# Decode to get output strings

decoded_output=tokenizer.decode(model_output[0], skip_special_tokens
=True, clean_up_tokenization_spaces=False)

print(decoded_output) # I am a boy
# Note that if your output language is not Hindi or Marathi, you should convert its script from Devanagari to the desired language using the Indic NLP Library.


I am a boy


# Preprocess the dataset

In [80]:
dataset = dataset.remove_columns(['id', 'Heading'])

In [81]:
# Context length of the model
from transformers import AutoConfig

# Load the model configuration
config = AutoConfig.from_pretrained("ai4bharat/IndicBART")

# Get the maximum sequence length
max_length = config.max_position_embeddings

print("Maximum sequence length:", max_length)

Maximum sequence length: 1024


In [82]:
def add_words(example):
    # Modify the column 'column_name' by adding words to each entry
    example['input'] = [entry + "  </s> <2hi>" for entry in example['Article']]
    example['target'] = ["<2hi> " + entry + " </s>" for entry in example['Summary']]
    return example

# Apply the function to the dataset using map
mod_dataset = dataset.map(add_words, batched=True, remove_columns=["Article", "Summary"])


In [91]:
max_input_length = 1024


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["input"],
        max_length=max_input_length,
        add_special_tokens=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    labels = tokenizer(
        examples["target"], truncation=True, add_special_tokens=True, return_tensors="pt", padding=True, max_length=max_input_length, 
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [92]:
tokenized_dataset = mod_dataset.map(preprocess_function, batched=True, batch_size=10)

NameError: name 'mod_dataset' is not defined

wandb: Network error (ReadTimeout), entering retry loop.


In [ ]:
tokenized_dataset = tokenized_dataset.remove_columns(
    mod_dataset["train"].column_names + ["token_type_ids"]
)

In [86]:
del dataset
del mod_dataset

# Evalutaion with ROUGE

In [60]:
import evaluate

rouge_score = evaluate.load("rouge")

In [61]:
import numpy as np
import indicnlp.tokenize.sentence_tokenize as tok

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(tok.sentence_split(pred.strip(), lang='hi', delim_pat='auto')) for pred in decoded_preds]
    decoded_labels = ["\n".join(tok.sentence_split(label.strip(), lang='hi', delim_pat='auto')) for label in decoded_labels]
    # Compute ROUGE scores
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels
    )
    # Extract the median scores
    print(result)
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

In [62]:
from datasets import load_metric

# Load the ROUGE metric
rouge_metric = load_metric("rouge")

# Example data: reference summaries and generated summaries
references = ["Reference summary 1", "Reference summary 2"]
hypotheses = ["Generated summary 1", "Generated summary 2"]

# Compute ROUGE scores
rouge_scores = rouge_score.compute(predictions=hypotheses, references=references)

# Print the scores
print(rouge_scores)


/home/shavak/anaconda3/envs/bart/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


{'rouge1': 0.6666666666666666, 'rouge2': 0.5, 'rougeL': 0.6666666666666666, 'rougeLsum': 0.6666666666666666}


## Creating a subset

In [63]:
from datasets import DatasetDict, Dataset

split_size = 0.1
sub_train_size = int(tokenized_dataset["train"].num_rows * split_size)
sub_test_size = int(tokenized_dataset["test"].num_rows * split_size)
sub_val_size = int(tokenized_dataset["validation"].num_rows * split_size)

train_subset = Dataset.from_dict(tokenized_dataset["train"].shuffle(seed=42)[:sub_train_size])
test_subset = Dataset.from_dict(tokenized_dataset["test"].shuffle(seed=42)[:sub_test_size])
val_subset = Dataset.from_dict(tokenized_dataset["validation"].shuffle(seed=42)[:sub_val_size])


In [64]:
print(train_subset)

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 795
})


In [65]:
subset_dataset = DatasetDict()
subset_dataset["train"] = train_subset
subset_dataset["test"] = test_subset
subset_dataset["validation"] = val_subset
subset_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 795
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 284
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 56
    })
})

In [66]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Wandb

In [ ]:
!pip install wandb
    

In [29]:
import wandb

In [30]:
run = wandb.init(project="ILSUM", config={
    'learning_rate': 5e-5,
    'architecture': 'IndicBART',
    "epochs": "8"
},
entity="parvp"
           )


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


wandb: Currently logged in as: parvp. Use `wandb login --relogin` to force relogin


## Generation Config

In [67]:
model.config.get_config_dict("ai4bharat/IndicBART")

({'activation_dropout': 0.1,
  'activation_function': 'gelu',
  'architectures': ['MBartForConditionalGeneration'],
  'attention_dropout': 0.1,
  'bos_token_id': 64000,
  'd_model': 1024,
  'classifier_dropout': 0.0,
  'decoder_attention_heads': 16,
  'decoder_ffn_dim': 4096,
  'decoder_layerdrop': 0.0,
  'decoder_layers': 6,
  'dropout': 0.1,
  'encoder_attention_heads': 16,
  'encoder_ffn_dim': 4096,
  'encoder_layerdrop': 0.0,
  'encoder_layers': 6,
  'eos_token_id': 64001,
  'gradient_checkpointing': False,
  'init_std': 0.02,
  'is_encoder_decoder': True,
  'max_position_embeddings': 1024,
  'model_type': 'mbart',
  'num_hidden_layers': 6,
  'pad_token_id': 0,
  'scale_embedding': False,
  'transformers_version': '4.3.2',
  'use_cache': True,
  'vocab_size': 64014,
  'tokenizer_class': 'AlbertTokenizer',
  '_commit_hash': '78466a0c0e29f9229f7005623ecd6bc4243c0ae0'},
 {})

In [68]:
from transformers import GenerationConfig
generation_config = GenerationConfig(
    max_new_tokens=512, 
    do_sample=True, 
    top_k=50,
    use_cache=True,
    num_beams=4,
    min_length=5,
    pad_token_id=pad_id,
    bos_token_id=bos_id, 
    eos_token_id=eos_id, 
    decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2hi>")   
)

generation_config.save_pretrained("config.json")



In [69]:
# Load the configuration from the saved file
loaded_config = GenerationConfig.from_pretrained("config.json")
print(loaded_config)
model.generation_config = loaded_config


GenerationConfig {
  "bos_token_id": 64000,
  "decoder_start_token_id": 64006,
  "do_sample": true,
  "eos_token_id": 64001,
  "max_new_tokens": 512,
  "min_length": 5,
  "num_beams": 4,
  "pad_token_id": 0
}



In [70]:
print(model.generation_config)

GenerationConfig {
  "bos_token_id": 64000,
  "decoder_start_token_id": 64006,
  "do_sample": true,
  "eos_token_id": 64001,
  "max_new_tokens": 512,
  "min_length": 5,
  "num_beams": 4,
  "pad_token_id": 0
}



In [71]:
bruh = AutoConfig.from_pretrained("ai4bharat/IndicBART")
print(config == bruh)

True


# Training

In [88]:
from transformers import Seq2SeqTrainingArguments

batch_size = 8
num_train_epochs = 8
# Show the training loss with every epoch
logging_steps = len(tokenized_dataset["train"])
model_name = "IndicBART"

args = Seq2SeqTrainingArguments(
    output_dir=f"{model_name}-finetuned-IndicSentenceSummarisation",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=True,
    report_to="wandb",
    run_name="first"
)


In [89]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/home/shavak/anaconda3/envs/bart/lib/python3.11/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [90]:
trainer.train()

  0%|          | 0/7960 [00:00<?, ?it/s]

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/home/shavak/anaconda3/envs/bart/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/72 [00:00<?, ?it/s]

{'rouge1': 0.11210098239975212, 'rouge2': 0.033911801925861684, 'rougeL': 0.11146325847556075, 'rougeLsum': 0.11116409464388373}
{'eval_loss': 0.40127116441726685, 'eval_rouge1': 11.2101, 'eval_rouge2': 3.3912, 'eval_rougeL': 11.1463, 'eval_rougeLsum': 11.1164, 'eval_runtime': 33.3499, 'eval_samples_per_second': 17.062, 'eval_steps_per_second': 2.159, 'epoch': 1.0}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'forced_eos_token_id': 2}
/home/shavak/anaconda3/envs/bart/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.

...

{'loss': 0.3884, 'grad_norm': 1.9990270137786865, 'learning_rate': 1.8844221105527637e-08, 'epoch': 8.0}


/home/shavak/anaconda3/envs/bart/lib/python3.11/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


  0%|          | 0/72 [00:00<?, ?it/s]

{'rouge1': 0.10831759043534087, 'rouge2': 0.033586619395055244, 'rougeL': 0.1069870424439844, 'rougeLsum': 0.10780701430965048}
{'eval_loss': 0.34586578607559204, 'eval_rouge1': 10.8318, 'eval_rouge2': 3.3587, 'eval_rougeL': 10.6987, 'eval_rougeLsum': 10.7807, 'eval_runtime': 30.9598, 'eval_samples_per_second': 18.379, 'eval_steps_per_second': 2.326, 'epoch': 8.0}
{'train_runtime': 4788.0587, 'train_samples_per_second': 13.295, 'train_steps_per_second': 1.662, 'train_loss': 0.3884411848969196, 'epoch': 8.0}


TrainOutput(global_step=7960, training_loss=0.3884411848969196, metrics={'train_runtime': 4788.0587, 'train_samples_per_second': 13.295, 'train_steps_per_second': 1.662, 'train_loss': 0.3884411848969196, 'epoch': 8.0})

In [41]:
x = subset_dataset["train"]


In [47]:
i = 5
print(x[i])
print(min(len(x[z]["input_ids"]) for z in range(0, 700)))

{'input_ids': [2638, 677, 6, 45805, 82, 563, 384, 85, 3140, 115, 4808, 142, 2725, 2605, 518, 33, 4295, 3188, 2775, 33, 415, 25253, 1958, 13, 577, 5620, 44, 803, 1361, 8, 687, 822, 10, 2807, 31, 836, 12462, 13, 1635, 214, 3472, 34, 1164, 2223, 213, 384, 22332, 2832, 13, 2716, 31, 1113, 11487, 59, 3191, 34154, 45, 18217, 8618, 8, 45805, 81, 984, 53388, 3857, 47, 115, 460, 36785, 55, 34, 2775, 31, 687, 822, 10, 2807, 45, 1488, 1849, 23072, 16, 346, 11369, 40, 1794, 373, 82, 563, 384, 85, 3140, 115, 4808, 142, 33, 19578, 33, 15, 8, 115, 460, 36785, 13, 633, 7, 8449, 430, 462, 56337, 34, 295, 7, 131, 8574, 16, 9021, 1657, 45, 2725, 518, 33, 1074, 33, 14460, 391, 20, 1958, 13, 577, 34154, 1768, 16424, 801, 33, 691, 11587, 2564, 8, 199, 5755, 72, 38105, 13, 186, 7, 285, 1113, 2547, 485, 33, 22489, 44, 4985, 15, 7, 3043, 285, 12462, 13, 17134, 13, 186, 43, 36180, 7913, 4830, 735, 257, 15, 8, 518, 5487, 7202, 13, 323, 14460, 391, 20, 31484, 45, 3863, 304, 13, 186, 45805, 81, 4584, 23153, 81, 25

In [ ]:
trainer.evaluate()

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.memory_allocated())

In [ ]:
inp = tokenizer("I am a boy </s> <2en>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids # tensor([[  466,  1981,    80, 25573, 64001, 64004]])

out = tokenizer("<2hi> मैं  एक लड़का हूँ </s>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids # tensor([[64006,   942,    43, 32720,  8384, 64001]])
# Note that if you use any language other than Hindi or Marathi, you should convert its script to Devanagari using the Indic NLP Library.
inp.to(device)
out.to(device)

model_outputs=model(input_ids=inp, decoder_input_ids=out[:,0:-1], labels=out[:,1:])

In [ ]:
string = "मुंबई इंडियंस ने आईपीएल 2024 के 14वें मैच में पहले बैटिंग करते हुए 20 ओवर में 9 विकेट खोकर 125 रन बनाए। राजस्थान रॉयल्स के गेंदबाजों के सामने मुंबई का टॉप-ऑर्डर फ्लॉप रहा। मुंबई के तीन बल्लेबाजों को ट्रेंट बोल्ट ने शून्य पर पवेलियन भेजा। हार्दिक और तिलक वर्मा के बीच अर्धशतकीय साझेदारी बनी। तिलक के बल्ले से 32 रन और हार्दिक ने 34 रन की पारी खेली। बोल्ट के अलावा राजस्थान के लिए चहल ने भी 3 विकेट चटकाए। इसके जवाब में 126 रन का पीछा करते हुए राजस्थान की टीम ने 15.3 ओवर में ही लक्ष्य हासिल कर लिया। राजस्थान की टीम की तरफ से रियाग पराग ने नाबाद 54 रन की पारी खेली, जिसमें 5 चौके और 3 छक्के शामिल रहे।"
inp = tokenizer(f"{string} </s> <2hi>", add_special_tokens=False, return_tensors="pt", padding=True).input_ids
inp = inp.to(model.device)
model_output=model.generate(inp, use_cache=True,
    min_length=30, #increase min length,
    num_return_sequences=1,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=1.0,
    num_beams=5,  # Use beam search
    repetition_penalty=2.0,  # Increase repetition penalty
    length_penalty=1.0,  # Adjust length penalty
    early_stopping=True,
    pad_token_id=pad_id,
    bos_token_id=bos_id,
    eos_token_id=eos_id,
    decoder_start_token_id=tokenizer._convert_token_to_id_with_added_voc("<2hi>"))


In [ ]:
decoded_output=tokenizer.decode(model_output[0], skip_special_tokens=True, clean_up_tokenization_spaces=False)
decoded_output

In [ ]:
x = tokenized_dataset["train"]

In [ ]:
x["input_ids"]

# Using Accelerator

In [ ]:
subset_dataset.set_format("torch")

In [ ]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    subset_dataset["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    subset_dataset["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [ ]:
from torch.utils.data import DataLoader

batch_size = 8
train_dataloader = DataLoader(
    subset_dataset["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=batch_size,
)
eval_dataloader = DataLoader(
    subset_dataset["validation"], collate_fn=data_collator, batch_size=batch_size
)

In [ ]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=1e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)
print(optimizer)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 20
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
import indicnlp.tokenize.sentence_tokenize as tok

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    preds = ["\n".join(tok.sentence_split(pred.strip(), lang='hi', delim_pat='auto')) for pred in decoded_preds]
    labels = ["\n".join(tok.sentence_split(label.strip(), lang='hi', delim_pat='auto')) for label in decoded_labels]

    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch
import numpy as np

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for step, batch in enumerate(train_dataloader):
        outputs = model(**batch)
        loss = outputs.loss
        record.append(loss)
        accelerator.backward(loss)
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for step, batch in enumerate(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                generation_config=generation_config
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]

            # If we did not pad to max length, we need to pad the labels too
            labels = accelerator.pad_across_processes(
                batch["labels"], dim=1, pad_index=tokenizer.pad_token_id
            )

            generated_tokens = accelerator.gather(generated_tokens).cpu().numpy()
            labels = accelerator.gather(labels).cpu().numpy()

            # Replace -100 in the labels as we can't decode them
            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(
                generated_tokens, skip_special_tokens=True
            )
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False)

            decoded_preds, decoded_labels = postprocess_text(
                decoded_preds, decoded_labels
            )

            rouge_score.add_batch(predictions=decoded_preds, references=decoded_labels)

    # Compute metrics
    result = rouge_score.compute()
    # Extract the median ROUGE scores
    wandb.log(result)
    result = {key: value * 100 for key, value in result.items()}
    result = {k: round(v, 4) for k, v in result.items()}
    print(f"Epoch {epoch}:", result, f"Loss: {loss}")

    # Save and upload
    """accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )"""

In [ ]:
x.append([y.cpu().detach().item() for y in record])

In [ ]:
z = np.array(x[-1])
z = np.transpose(z)
z.size

In [ ]:
import matplotlib.pyplot as plt


# Create x-axis values (epochs or steps)
epochs = range(1, len(record) + 1)

# Plot the loss
plt.plot(epochs, z, 'b', label='Training loss')
plt.title('Training Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in list(
                          locals().items())), key= lambda x: -x[1]):
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))


In [ ]:
import sys

local_vars = list(locals().items())
for var, obj in local_vars:
    print(var, sys.getsizeof(obj))
